<a href="https://colab.research.google.com/github/sushilmanda/Google-collab/blob/master/job.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **2) Write a function in python that take dataframe as input and drop columns having Pearson correlation more than 0.85**


### Assumptions
* We do not want to loose original feature (So PCA can not be used) 

#### I will use VIF to remove the features

 Objective: Remove features havingcorrelation more than 0.85 (drop least amount  of variable as possible.)

VIF calculation for Pearson correlation > 0.85

>$VIF=1/(1-R^2)$

here R = Pearson correlation(should be greater than 0.85 as per problem)
          
>$VIF < 1/(1-0.85^2)$

>$VIF < 3.6$

In [31]:
# Importing required libraries
import pandas as pd, numpy as np 
from statsmodels.stats.outliers_influence import variance_inflation_factor

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## **Main Function**

In [160]:
def Feature_selection(x): # Function
    i = 0
    while i < len(x.columns):
      vif = pd.DataFrame()
      vif['Features'] = x.columns
      vif['VIF'] = [variance_inflation_factor(x.values, i) for i in range(x.shape[1])]
      vif = vif.sort_values(by = "VIF", ascending = False)
      if vif.iloc[0,1] > 3.6:
        x = x.drop(vif.iloc[0,0], axis = 1)
      i = i+1
    return x

**Now let's test our function**

In [165]:
# dataset for testig
data = {'Gender': [0,1,1,0,0],
        'Age': [27,26,26,27,26],
        'Ear of service': [1.7,1.1,1.2,1.6,1.5],
        'Graduate': [0,1,0,1,1],
        'Salery': [39343,43205,47731,46525,40891]
        }

df = pd.DataFrame(data,columns=['Gender','Age','Ear of service','Graduate','Salery'])

In [17]:
# Creating correlation metrix
df.corr()

,Gender,Age,Ear of service,Graduate,Salery
Gender,1.000000,-0.666667,-0.952217,-0.166667,0.492103
Age,-0.666667,1.000000,0.811147,-0.166667,-0.154340
Ear of service,-0.952217,0.811147,1.000000,-0.105802,-0.449290
Graduate,-0.166667,-0.166667,-0.105802,1.000000,0.000510
Salery,0.492103,-0.154340,-0.449290,0.000510,1.000000


From table we can observe some of the attributes have more than 0.85 Pearson correlation

In [166]:
# Lets run our function on df dataset
d = Feature_selection(df)
d.head()

,Gender,Ear of service,Graduate
0,0,1.7,0
1,1,1.1,1
2,1,1.2,0
3,0,1.6,1
4,0,1.5,1


In [167]:
d.corr()

,Gender,Ear of service,Graduate
Gender,1.000000,-0.952217,-0.166667
Ear of service,-0.952217,1.000000,-0.105802
Graduate,-0.166667,-0.105802,1.000000


 Gender and Ear of service have correlation of 0.95 but still not droped 

Let's check VIF, by this we know why they are not dropped

In [173]:
vif = pd.DataFrame()
vif['Features'] = df.columns
vif['VIF'] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif1 = pd.DataFrame()
vif1['Features'] = d.columns
vif1['VIF'] = [variance_inflation_factor(d.values, i) for i in range(d.shape[1])]
vif1['VIF'] = round(vif1['VIF'], 2)
vif1 = vif1.sort_values(by = "VIF", ascending = False)
print("VIF for original Data \n", vif)
print('\n')
print("VIF for Transformed Data \n", vif1)

VIF for original Data 
          Features       VIF
1             Age  17374.58
2  Ear of service  13358.91
0          Gender    480.07
3        Graduate     50.99
4          Salery      1.49


VIF for Transformed Data 
          Features   VIF
1  Ear of service  2.60
2        Graduate  2.32
0          Gender  1.35


From this table we know that 'Ear of service' have 2.6 VIF. But what does this mean, lets see

> $VIF = 1/(1-R^2)$

by this,

> $ 2.6 = 1/(1-R^2)$

> $ R = 0.784$

From here it can be seen the even if 'Gender' and 'Ear of service' have more than 95% correlation but only 78.4% of 'Ear of service' column can be justified by other columns so that why this column was not deleted.

# **1) Write a function in python that inputs a dataframe and identify which columns have date in them.**

### Assumption

* Input Dataframe have all the features dtype as object(string) or integers.

## **Main Function**

In [104]:
def Date_Filter(x):
  import pandas as pd # importing librar needed for function
  new = pd.DataFrame() 
  for col in x.columns: # This loop will give us a new DataFrame named "new" having only dateds column of input DataFrame
    if x[col].dtype == 'object':
        try:
            new[col] = pd.to_datetime(x[col])
        except ValueError:
            pass
  co = new.columns
  for i in co: # This loop will make features containing original date columns and new columns which are difference between these columns taking 2 at a time.
    for k in co:
      if i<k:
        new[i+k] = abs(new[i].sub(new[k], axis = 0)) # Taking absolute value of subraction because subraction might give negative values also
  return new

###### Salmple DataFrame

In [109]:
import pandas as pd
#creating the test data
df = pd.DataFrame({'date1': ['2015/12/27','2015-01-28', '2015-12-29','2018-5-30','2017-1-1','Mar/12/2025'], 
                   'date2': ['2016/10/2','2010-01-08', '2018-12-29','2018-5-Jan','2017-11-10','03/10/2025'],
                   'date3': ['2014/12/21','2012-11-18', '2017-1-29','2018-5-09','2017-9-1','08/10/2025'],
                   'Names': ['a','b','c','d','e','f'],
                   'Number': [21,22,23,24,25,26]})
print(df.info())
print("\n")
print(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date1   6 non-null      object
 1   date2   6 non-null      object
 2   date3   6 non-null      object
 3   Names   6 non-null      object
 4   Number  6 non-null      int64 
dtypes: int64(1), object(4)
memory usage: 368.0+ bytes
None


         date1       date2       date3 Names  Number
0   2015/12/27   2016/10/2  2014/12/21     a      21
1   2015-01-28  2010-01-08  2012-11-18     b      22
2   2015-12-29  2018-12-29   2017-1-29     c      23
3    2018-5-30  2018-5-Jan   2018-5-09     d      24
4     2017-1-1  2017-11-10    2017-9-1     e      25
5  Mar/12/2025  03/10/2025  08/10/2025     f      26


We can see date1, date2 and date3 dtype are as objects

###### Now, Let's run over function

In [113]:
print(Date_Filter(df).info(), '\n')
Date_Filter(df)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype          
---  ------      --------------  -----          
 0   date1       6 non-null      datetime64[ns] 
 1   date2       6 non-null      datetime64[ns] 
 2   date3       6 non-null      datetime64[ns] 
 3   date1date2  6 non-null      timedelta64[ns]
 4   date1date3  6 non-null      timedelta64[ns]
 5   date2date3  6 non-null      timedelta64[ns]
dtypes: datetime64[ns](3), timedelta64[ns](3)
memory usage: 416.0 bytes
None 



,date1,date2,date3,date1date2,date1date3,date2date3
0,2015-12-27,2016-10-02,2014-12-21,280 days,371 days,651 days
1,2015-01-28,2010-01-08,2012-11-18,1846 days,801 days,1045 days
2,2015-12-29,2018-12-29,2017-01-29,1096 days,397 days,699 days
3,2018-05-30,2018-01-05,2018-05-09,145 days,21 days,124 days
4,2017-01-01,2017-11-10,2017-09-01,313 days,243 days,70 days
5,2025-03-12,2025-03-10,2025-08-10,2 days,151 days,153 days


### We have have our output table nake as new

In [114]:

new.head()

,date1,date2,date3,date1date2,date1date3,date2date3
0,2015-12-27,2016-10-02,2014-12-21,280 days,371 days,651 days
1,2015-01-28,2010-01-08,2012-11-18,1846 days,801 days,1045 days
2,2015-12-29,2018-12-29,2017-01-29,1096 days,397 days,699 days
3,2018-05-30,2018-01-05,2018-05-09,145 days,21 days,124 days
4,2017-01-01,2017-11-10,2017-09-01,313 days,243 days,70 days


Note: This function can work on different date formats but can not be used on Invalid entries or it will give error.
